# Forest Cover Type Prediction

## 1. Setup

### 1.1 Global Variables 

In [ ]:
RANDOM_SEED = 0
NUM_FOLDS = 12

In [ ]:
import numpy as np
import pandas as pd
import math
import scipy
import time
import gc
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt


# Model & Evaluation
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer, PowerTransformer
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn import metrics, ensemble,linear_model
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

### 1.2 Dataset

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sampleSubmission.csv')

In [ ]:
df = pd.concat([test.assign(indic="test"), train.assign(indic="train")]).set_index("Id")

In [ ]:
df.tail(5)

## 2. Exploratory Data Analysis (descriptive analytics) 

### 2.1 Basic understanding

Due to our little understanding of trees and nature, we decided not to use any automated exploratory data analysis tool for data exploration. We believe that a manual approach would better understand the variables and their relationship between them and the target. 

In this very first step, we can see that all our variables are numerical and maybe, most of them are already One-Hot-Encoded. Furthermore, we ca see that there are 561012 rows. 

In [ ]:
df.dtypes

In [ ]:
len(df)

After running a statistical analysis, we can see that vertical distance to hydrology have negatives values, which means that we can't use some tools such as the chi-square test and log transformations. 

In [ ]:
df.describe()

We can see that there are columns with just two unique values, which means that there are categorical values. We will check it below. 

In [ ]:
df.nunique()

### 2.2 Null values in the data set


There are no values in the dataset. 

In [ ]:
df.isnull().any()

### 2.3 Number of zeros in all columns 


There are several zero values in the dataset. For some features, it could be expected. Aspect is the degrees can be 0 and some measures such as horizontals and vertical distances. However, even if zero values can be possible, further exploration of the dataset and a more profound investigation of the topic, to be sure that those zero values are not errors.

In [ ]:
for variable in df.iloc[:,0:11]:
    column = df[variable] 
    count = (column == 0).sum()
    print('number of zeros in column ', variable, ' is:',count)

### 2.4 Checking if binary variables are really binary


We can see that the features with two values are one-hot encoded features. All of them are 0 or 1. However, soil_7 has only 105 values, soil_15 has 3 and soil_36 only 119.

In [ ]:
for i in df.iloc[:,10:55]:
    print(df[i].value_counts())

### 2.5 Analysing continous numerical variables 

##### Checking for Outliers

##### Finding outliers 

We can see that our dataset has several outliers. Sometimes, it can negatively affect our data cleaning or our prediction. However, the presence of outliers can also tell us a lot about our data's behavior and help us to understand our work better.

In [ ]:
def find_outliers_IQR(df):
    q1=df.quantile(0.25)
    q3=df.quantile(0.75)
    IQR=q3-q1
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
    return outliers

In [ ]:
find_outliers_IQR(df.iloc[:,1:11]).count()

In [ ]:
for column in df.iloc[:,1:11]:
    plt.figure()
    df.boxplot([column])

### 2.6 Variables Distribution 

#### Distributon of the Predicted variable

Looking at this graph, we can see that we have almost the same quantity of values for every type of cover forest, which is the target variable. Having a balanced distribution between the categories helps our models to have a better prediction. 

In [ ]:
sns.set(rc = {'figure.figsize':(10,6)})

sns.histplot(df["Cover_Type"], kde=False, color ='green')

plt.xlabel("Cover_Type")
plt.ylabel("Number")
plt.title("Distribution of Cover Types")
plt.legend(["Count"]);

### 2.7 Correlation between numerical variables

Looking at the correlation matrix, we can see two correlated variables,  hillshade 9 am and hillshade 3 pm. 

In [ ]:
plt.subplots(figsize=(12,7))
sns.heatmap(df.iloc[:,0:10].corr(), annot=True, cmap = 'flare');

### 2.8 Undoing the one-hot encoder

In [ ]:
def wilderness_area_encoding(df):
    data = df.copy()
    data['Wilderness_Area'] = 0
    for i in range(1,5):
        data['Wilderness_Area'] += i*data[f'Wilderness_Area{i}']
    return data

In [ ]:
def soil_type_encoding(df):
    data = df.copy()
    data['Soil_Type'] = 0
    for i in range(1,41):
        data['Soil_Type'] += i*data[f'Soil_Type{i}']
    return data

In [ ]:
df= soil_type_encoding(df)

In [ ]:
df = wilderness_area_encoding(df)

###  Number of covert_type by wilderness_area

In [ ]:
plt.subplots(figsize=(12,7))
sns.countplot(x = 'Wilderness_Area', hue = 'Cover_Type', data = df,palette = "Greens" )
plt.show()

### 2.9 Numerical Continious Variables 
We thought it would be interesting to know the relationship betweent the numerical continuious variables 

In [ ]:
# We can see that elevation is the most important feature in our dataset, followed by vertical and horizontal distance to hydrology and fire points. It could be interesting to know the relationships of those variables. 
fg, ax = plt.subplots(nrows=2, ncols=2,figsize=(15,10), sharex=True,)
sns.scatterplot(data=df, x="Elevation", y="Horizontal_Distance_To_Roadways",  color= 'green', ax=ax[0,0])
sns.scatterplot(data=df, x="Elevation", y="Horizontal_Distance_To_Fire_Points",  color= 'green', ax=ax[0,1])
sns.scatterplot(data=df, x="Elevation", y="Horizontal_Distance_To_Hydrology",  color= 'green', ax=ax[1,0])
sns.scatterplot(data=df, x="Elevation", y="Vertical_Distance_To_Hydrology",  color= 'green', ax=ax[1,1]);  

As we can see there is a linear relationship between this variables. That can help us to come up with new variables that can help to improve the model's accuracy. 

### 2.10 Distribution of Variables 

Continous variables are not normaly distributed.

In [ ]:
for i in df.iloc[:,1:10]:
    #sns.displot(df[i], label= i)
    sns.displot(df[i], label= i, kind="kde")

## 3. Data Cleaning

### 3.1 Scaling
We will be scaling it the pipeline 

We will try Robust and Standard scaler 

### 3.2 Normalising
We will be normalising in the pipeline

We will try PowerTransformer 

### 3.3 Outliers 

We decided to keep them since we don't have enough knowledge about the data and what is actually considered an outlier or not 

## 4. Feature Engineering

### 4.1 Add General Features

In [ ]:
# 
def add_general_features(data):
    df = data.copy()

            
    df['Horizontal_Distance_To_Roadways_Log'] = [math.log(v+1) for v in df['Horizontal_Distance_To_Roadways']]
    df['Water Elevation'] = df['Elevation'] - df['Vertical_Distance_To_Hydrology']
    df['Hydro_Fire_1'] = df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Fire_Points']
    df['Hydro_Fire_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Road_1'] = abs(df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Roadways'])
    df['Hydro_Road_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_1'] = abs(df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_2'] = abs(df['Horizontal_Distance_To_Fire_Points'] - df['Horizontal_Distance_To_Roadways'])
    df['EHiElv'] = df['Horizontal_Distance_To_Roadways'] * df['Elevation']
    df['EVDtH'] = df.Elevation - df.Vertical_Distance_To_Hydrology
    df['EHDtH'] = df.Elevation - df.Horizontal_Distance_To_Hydrology * 0.2
    df['Elev_3Horiz'] = df['Elevation'] + df['Horizontal_Distance_To_Roadways']  + df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Hydrology']
    df['Elev_Road_1'] = df['Elevation'] + df['Horizontal_Distance_To_Roadways']
    df['Elev_Road_2'] = df['Elevation'] - df['Horizontal_Distance_To_Roadways']
    df['Elev_Fire_1'] = df['Elevation'] + df['Horizontal_Distance_To_Fire_Points']
    df['Elev_Fire_2'] = df['Elevation'] - df['Horizontal_Distance_To_Fire_Points']
    
    # Fill NA
    df.fillna(0, inplace = True)

    
    return df

In [ ]:
df = add_general_features(df)

### 4.2 Soil Type Features
new features based on the soil-type variables:


1. Climatic Zone
2. Geologic Zone
3. Surface Cover
4. Rock Size
5. Interaction Terms


### ELU Codes
The soil type number is based on the USFS Ecological Landtype Units (ELUs). 

In [ ]:
ELU_CODE = {
    1:2702,2:2703,3:2704,4:2705,5:2706,6:2717,7:3501,8:3502,9:4201,
    10:4703,11:4704,12:4744,13:4758,14:5101,15:5151,16:6101,17:6102,
    18:6731,19:7101,20:7102,21:7103,22:7201,23:7202,24:7700,25:7701,
    26:7702,27:7709,28:7710,29:7745,30:7746,31:7755,32:7756,33:7757,
    34:7790,35:8703,36:8707,37:8708,38:8771,39:8772,40:8776
}

### 4.2.1  Climatic Zone (Ordinal Variable)

1. lower montane dry
2. lower montane
3. montane dry
4. montane
5. montane dry and montane
6. montane and subalpine
7. subalpine
8. alpine

In [ ]:
def climatic_zone(df):
    data = df.copy()
    data['Climatic_Zone'] = df['Soil_Type'].apply(
        lambda x: int(str(ELU_CODE[x])[0])
    )
    return data

In [ ]:
df = climatic_zone(df)

### 4.2.2  Geologic Zone (Nominal Variable)

1. alluvium
2. glacial
3. shale
4. sandstone
5. mixed sedimentary
6. unspecified in the USFS ELU Survey
7. igneous and metamorphic
8. volcanic

In [ ]:
def geologic_zone(df):
    data = df.copy()
    data['Geologic_Zone'] = df['Soil_Type'].apply(
        lambda x: int(str(ELU_CODE[x])[1])
    )
    return df

In [ ]:
# Geologic Zone
df = geologic_zone(df)

### 4.2.3 Surface Cover (Ordinal Variable)

1. (Stony/Bouldery) 
2. (Very Stony/Very Bouldery) 
3. (Extremely Stony/Extremely Bouldery) 
4. (Rubbly)
5. (Very Rubbly) 

If no description of the surface cover is given, we give it a value of 0.

In [ ]:
def surface_cover(df):
    #Group IDs
    no_desc = [7,8,14,15,16,17,19,20,21,23,35]
    stony = [6,12]
    very_stony = [2,9,18,26]
    extremely_stony = [1,22,24,25,27,28,29,30,31,32,33,34,36,37,38,39,40]
    rubbly = [3,4,5,10,11,13]

    #Create dictionary
    surface_cover = {i:0 for i in no_desc}
    surface_cover.update({i:1 for i in stony})
    surface_cover.update({i:2 for i in very_stony})
    surface_cover.update({i:3 for i in extremely_stony})
    surface_cover.update({i:4 for i in rubbly})
    
    #Create Feature
    data = df.copy()
    data['Surface_Cover'] = df['Soil_Type'].apply(
        lambda x: surface_cover[x]
    )
    return data

In [ ]:
#Surface cover
df = surface_cover(df)

### 4.2.4 Rock Size (Nominal)

1. Stones
2. Boulders
3. Rubble

If the soil type description has no mention of rock size, we give it a default value of 0.

In [ ]:
def rock_size(df):
    
    # Group IDs
    no_desc = [7,8,14,15,16,17,19,20,21,23,35]
    stones = [1,2,6,9,12,18,24,25,26,27,28,29,30,31,32,33,34,36,37,38,39,40]
    boulders = [22]
    rubble = [3,4,5,10,11,13]

    # Create dictionary
    rock_size = {i:0 for i in no_desc}
    rock_size.update({i:1 for i in stones})
    rock_size.update({i:2 for i in boulders})
    rock_size.update({i:3 for i in rubble})
    
    data = df.copy()
    data['Rock_Size'] = df['Soil_Type'].apply(
        lambda x: rock_size[x]
    )
    return data

In [ ]:
#Rock size
df = rock_size(df)

### 4.2.5 Soil Type Interactions
We include only those features which resulted in improved CV accuracy.

In [ ]:
def soiltype_interactions(data):
    df = data.copy()
            
    # Important Soil Types
    df['Soil_12_32'] = df['Soil_Type32'] + df['Soil_Type12']
    df['Soil_Type23_22_32_33'] = df['Soil_Type23'] + df['Soil_Type22'] + df['Soil_Type32'] + df['Soil_Type33']
    
    # Soil Type Interactions
    df['Soil29_Area1'] = df['Soil_Type29'] + df['Wilderness_Area1']
    df['Soil3_Area4'] = df['Wilderness_Area4'] + df['Soil_Type3']
    
    #  New Feature Interactions
    df['Climate_Area2'] = df['Wilderness_Area2']*df['Climatic_Zone'] 
    df['Climate_Area4'] = df['Wilderness_Area4']*df['Climatic_Zone'] 
    df['Rock_Area1'] = df['Wilderness_Area1']*df['Rock_Size']    
    df['Rock_Area3'] = df['Wilderness_Area3']*df['Rock_Size']  
    df['Surface_Area1'] = df['Wilderness_Area1']*df['Surface_Cover'] 
    df['Surface_Area2'] = df['Wilderness_Area2']*df['Surface_Cover']   
    df['Surface_Area4'] = df['Wilderness_Area4']*df['Surface_Cover'] 
    
    # Fill NA
    df.fillna(0, inplace = True)
    
    return df

In [ ]:
#Soil type interactions
df = soiltype_interactions(df)

In [ ]:
plt.subplots(figsize=(40,30))
sns.heatmap(df.corr(), annot=True, cmap = 'flare');

### 4.3 Dimension Reduction
We drop the original soil features since the new features discribe them.

In [ ]:
soil_features = [f'Soil_Type{i}' for i in range(1,41)]

In [ ]:
df.drop(columns = soil_features, inplace = True)

In [ ]:
wilderness_features = [f'Wilderness_Area{i}' for i in range(1,5)]

In [ ]:
df.drop(columns = wilderness_features, inplace = True)

### Correlations after feautre engineering  
There are still many correlated features! but some algorithms can help to deal with this problem.
Since in the real world, we deal with a big quantity of feauters that can't be picked manually, we wanted to test all those features together.

In [ ]:
plt.subplots(figsize=(30,30))
sns.heatmap(df.corr(), annot=True, cmap = 'flare');

## 5. Model Training 

### 5.1 Split dataframe 
Here we are splitting our dataframe back to the train and test based on the ind 

In [ ]:
test, train = df[df["indic"].eq("test")],df[df["indic"].eq("train")]

In [ ]:
X = train.drop(['Cover_Type','indic'],axis=1)
X_test = test.drop(['Cover_Type', 'indic'],axis=1)
y = train.Cover_Type

### 5.2 Multinomial Logistic Regression

In [ ]:
# Function to print the most important features of a logreg classifier based on the coefficient values
def get_feature_importance(clf, feature_names):
    return pd.DataFrame({'variable': feature_names, # Feature names
                         'coefficient': clf.coef_[0] # Feature Coeficients
                    }) \
    .round(decimals=2) \
    .sort_values('coefficient', ascending=False) \
    .style.bar(color=['red', 'green'], align='zero')

#### 5.2.1 Ridge Reguralized

In [ ]:
pipeline = Pipeline(steps=[('scaler', RobustScaler()), ('model', Ridge(alpha=1.0))])

#Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {}

skf = StratifiedKFold(n_splits = 12, shuffle = True, random_state = 0)

# Create a grid search to try all the possible number of PCs
estimator = GridSearchCV(pipeline, param_grid, cv=skf)
estimator.fit(X, y)
estimator.best_score_

In [ ]:
ridge_mod = linear_model.LogisticRegression(max_iter=10000,penalty='l2')
print("Accuracy = {:.4}".format(np.mean(cross_val_score(ridge_mod, X, y, cv=skf))))

In [ ]:
get_feature_importance(ridge_mod.fit(X,y), X.columns.get_level_values(0).tolist())

In [ ]:
ridge_mod = linear_model.LogisticRegression(max_iter=10000,penalty='l2')
alphas = 10**np.linspace(-1,-4,100)

coefs_ = [] 
scores_ = [] 

# Go over the regularization values list defined above, train a logreg model for each of the regularization values and evaluate it.
for a in alphas:
    ridge_mod.set_params(C=a) # Set the regularization parameter 
    scores_.append(np.mean(cross_val_score(ridge_mod, X, y, cv=skf))) # Appends the accuracy of the model
    coefs_.append(ridge_mod.fit(X, y).coef_.ravel().copy()) # Appends the coefficient of the model

# Conver the coefficient and scores arrays to numpy arrays
coefs_ = np.array(coefs_)
scores_ = np.array(scores_)

# Define the figures to plot the values
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
fig.suptitle('Logistic Regression Path', fontsize=20)

# Coeff Weights Plot
ax1.plot(alphas, coefs_, marker='o')
ymin, ymax = plt.ylim()
ax1.set_ylabel('Coefficient Weights', fontsize = 15)
ax1.set_xlabel('Alpha', fontsize = 15)
ax1.axis('tight')

# Accuracy Plot
ax2.plot(alphas, scores_, marker='o')
ymin, ymax = plt.ylim()
ax2.set_ylabel('Accuracy Score', fontsize = 15)
ax2.set_xlabel('Alpha', fontsize = 15)
ax2.axis('tight')

plt.show()

In [ ]:
from sklearn.feature_selection import SelectFromModel
alphas = 10**np.linspace(-1,-4,100)

ridge_mod_cv = linear_model.LogisticRegressionCV(max_iter=10000,penalty='l2',Cs=alphas)
print("Accuracy = {:.4}".format(np.mean(cross_val_score(ridge_mod_cv, X, y, cv=skf))))

####  5.2.2 Lasso Reguralized
Ridge gave us a score of 0.3467. We Will see if Lasso (which actually removes features by making their coefficients equal to 0) improves the unregularized model.

In [ ]:
pipeline = Pipeline(steps=[('scaler', RobustScaler()), ('normaliser', PowerTransformer()), ('model', linear_model.LogisticRegression())])

#Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {'model__penalty':['l1'], 'model__solver':['liblinear']}

skf = StratifiedKFold(n_splits = 12, shuffle = True, random_state = 0)

# Create a grid search to try all the possible number of PCs
estimator = GridSearchCV(pipeline, param_grid, cv=skf)
estimator.fit(X, y)
estimator.best_score_

In [ ]:
lasso_mod = linear_model.LogisticRegression(penalty='l1', solver='liblinear')
print("Accuracy = {:.4}".format(np.mean(cross_val_score(lasso_mod, X, y, cv=skf))))

In [ ]:
get_feature_importance(lasso_mod.fit(X,y), X.columns.get_level_values(0).tolist())

In [ ]:
lasso_mod = linear_model.LogisticRegression(penalty='l1',solver='liblinear')
alphas = 10**np.linspace(-1,-4,100)

coefs_ = []
scores_ = []
for a in alphas:
    lasso_mod.set_params(C=a)
    scores_.append(np.mean(cross_val_score(lasso_mod, X, y, cv=5))) # Appends the accuracy of the model
    coefs_.append(lasso_mod.fit(X, y).coef_.ravel().copy()) # Appends the coefficient of the model

coefs_ = np.array(coefs_)
scores_ = np.array(scores_)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
fig.suptitle('Logistic Regression Path', fontsize=20)

# Coeff Weights Plot
ax1.plot(alphas, coefs_, marker='o')
ymin, ymax = plt.ylim()
ax1.set_ylabel('Coefficient Weights', fontsize = 15)
ax1.set_xlabel('log(C)', fontsize = 15)
ax1.axis('tight')

# Accuracy Plot
ax2.plot(alphas, scores_, marker='o')
ymin, ymax = plt.ylim()
ax2.set_ylabel('Accuracy Score', fontsize = 15)
ax2.set_xlabel('log(C)', fontsize = 15)
ax2.axis('tight')

plt.show()

In [ ]:
from sklearn.feature_selection import SelectFromModel

lasso_mod_cv = linear_model.LogisticRegressionCV(max_iter=10000,penalty='l1',solver='liblinear',Cs=alphas)
print("Accuracy = {:.4}".format(np.mean(cross_val_score(lasso_mod_cv, X, y, cv=skf))))

In [ ]:
lasso_mod_cv.fit(X,y)
model = SelectFromModel(X, prefit=True)
X_new = model.transform(X)
print("Original Number of Features = {} --> Number of features selected by Lasso = {}".format(X.shape[1], X_new.shape[1]))

In [ ]:
reduced_lasso_mod = linear_model.LogisticRegression(max_iter=10000,penalty='l1', solver='liblinear')
print("Accuracy = {:.4}".format(np.mean(cross_val_score(reduced_lasso_mod, X_new, y, cv=5))))

In [ ]:
get_feature_importance(reduced_lasso_mod.fit(X_new,y), X.columns[model.get_support()].get_level_values(0).tolist())

Lasso gave us a score of 0.6909! better than ridge and with less features!

### 5.3 Tree Based Methods 

#### 5.3.1 Gradient Boosting
- uses regulaiazed linear models 
- stores data on a data on a data structure called DMatrix for faster iteration

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

pipeline = Pipeline(steps=[('scaler', RobustScaler()), ('model', GradientBoostingRegressor())])

#Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {'model__max_features':[20,23,25], 'model__min_samples_leaf':[1,2], 'model__n_estimators':[500,1000]}

skf = StratifiedKFold(n_splits = 12, shuffle = True, random_state = 0)

# Create a grid search to try all the possible number of PCs
estimator = GridSearchCV(pipeline, param_grid, cv=skf)
estimator.fit(X, y)
estimator.best_score_

#### 5.3.2 Random Forest

In [ ]:
pipeline = Pipeline(steps=[('scaler', RobustScaler()), ('model', RandomForestClassifier())])

#Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {'model__n_estimators':[200], 'model__n_jobs':[-1]}

skf = StratifiedKFold(n_splits = 12, shuffle = True, random_state = 0)

# Create a grid search to try all the possible number of PCs
estimator = GridSearchCV(pipeline, param_grid, cv=skf)
estimator.fit(X, y)
estimator.best_score_

In [ ]:
#taking code from the class practice

In [ ]:
boston_bagging = RandomForestRegressor(random_state=42, max_features=len(X.columns))
print("Accuracy = {0:.4f}".format(-np.mean(cross_val_score(boston_bagging, X, y, scoring='accuracy'))))

In [ ]:
plt.figure(figsize=(10,10))
boston_bagging.fit(X,y)
plt.bar(X.columns, boston_bagging.feature_importances_)
plt.title('Feature Importance', fontsize=16);

The performance is similar with way less features! 

In [ ]:
boston_rf = RandomForestRegressor(random_state=42, max_features='sqrt')
print("MSE = {0:.4f}".format(-np.mean(cross_val_score(boston_rf, X, y, scoring='neg_mean_squared_error'))))

In [ ]:
plt.figure(figsize=(10,10))
boston_rf.fit(X,y)
plt.bar(X.columns, boston_rf.feature_importances_)
plt.title('Feature Importance', fontsize=16);

In [ ]:
#final based on our results we chose the following important features features 
boston_rf_small = RandomForestRegressor(random_state=42, max_features='sqrt')
print("MSE = {0:.4f}".format(-np.mean(cross_val_score(boston_rf, X[['Elevation', 'Soil_Type','Water_Elevation', 'EVDtHb', 'EVDtH', 'EHDtH', 'Climatic_Zone']], y, scoring='neg_mean_squared_error'))))

#### 5.3.3 Extremely Randomized Trees (Extra Trees)
two main differences with other tree based sembsle methods are:
1. it splits nodes b y choosing cut-points fully at random 
2. it uses the whole learning sample (rather thana bootstrap replica) to grow the trees 

##### Testing on our raw data 

In [ ]:
X1 = pd.read_csv('data/train.csv').drop(['Cover_Type'],axis=1)
X_test = pd.read_csv('data/test.csv')
y1 = train.Cover_Type

In [ ]:
# Pipeline the entire process: Scale the data -> ExtraTreesClassifier
pipeline = Pipeline(steps=[('scaler', RobustScaler()), ('model', ExtraTreesClassifier())])

#Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {'model__n_estimators': [116],'model__min_samples_split':[2], 
             'model__max_features':[14],'model__random_state':[42], 'model__n_jobs': [-1]}

skf = StratifiedKFold(n_splits = 12, shuffle = True, random_state = 0)

# Create a grid search to try all the possible number of PCs
estimator = GridSearchCV(pipeline, param_grid, cv=skf)
estimator.fit(X1, y1)
print(estimator.best_params_)
print(estimator.best_score_)

##### Testing on our cleaned, & featured engineered data 

In [ ]:
# Pipeline the entire process: Scale the data -> ExtraTreesClassifier
pipeline = Pipeline(steps=[('scaler', RobustScaler()), ('model', ExtraTreesClassifier())])

#Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {'model__n_estimators': [115,116,117,118],'model__min_samples_split':[2], 
             'model__max_features':[14,15,16],'model__random_state':[0,42], 'model__n_jobs': [-1]}

skf = StratifiedKFold(n_splits = 12, shuffle = True, random_state = 0)

# Create a grid search to try all the possible number of PCs
estimator = GridSearchCV(pipeline, param_grid, cv=skf)
estimator.fit(X, y)
print(estimator.best_params_)
print(estimator.best_score_)

### 5.4 Saving to CSV 

In [ ]:
# Submission with feature engineering
submission['Cover_Type'] = estimator.best_estimator_.predict(X_test).astype("int")
submission.to_csv('GroupH_submission.csv', index=False)

ExtraTreesClassifier Accuracy: 0.9124

Kaggle Accuracy: 0.81942

Kaggle Account: Tamara Samaha